<a href="https://colab.research.google.com/github/jarrin-kevin/nuevo/blob/master/Primera_red_neuronal_con_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neuronales recurrentes para procesamiento de lenguaje natural


### Importar librerias necesarias

In [80]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time

### Cargar el dataset

In [81]:
from google.colab import files
files.upload()

Saving fragmento-de-libro.txt to fragmento-de-libro (3).txt


{'fragmento-de-libro (3).txt': b"That, poor contempt, or claim'd thou slept so faithful,\r\nI may contrive our father; and, in their defeated queen,\r\nHer flesh broke me and puttance of expedition house,\r\nAnd in that same that ever I lament this stomach,\r\nAnd he, nor Butly and my fury, knowing everything\r\nGrew daily ever, his great strength and thought\r\nThe bright buds of mine own.\r\n\r\nBIONDELLO:\r\nMarry, that it may not pray their patience.'\r\n\r\nKING LEAR:\r\nThe instant common maid, as we may less be\r\na brave gentleman and joiner: he that finds us with wax\r\nAnd owe so full of presence and our fooder at our\r\nstaves. It is remorsed the bridal's man his grace\r\nfor every business in my tongue, but I was thinking\r\nthat he contends, he hath respected thee.\r\n\r\nBIRON:\r\nShe left thee on, I'll die to blessed and most reasonable\r\nNature in this honour, and her bosom is safe, some\r\nothers from his speedy-birth, a bill and as\r\nForestem with Richard in your he

In [82]:
path_to_fileDL ='/content/fragmento-de-libro.txt' #guardo en una variable la ruta del archivo que se guardo en collab
text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8') #abro la ruta del archivo con la funcion open, y lee el archivo en modo binario y lo codifico en utf8 para que sea legible
print('Longitud del texto: {} carácteres'.format(len(text))) #imprimo la longitud del texto
vocab = sorted(set(text)) #con la funcion sorted ordeno, y con la funcion set obtengo todos los caracteres que se involucran en el texto
print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab) # imprimo los caracteres del texto


Longitud del texto: 103289 carácteres
El texto está compuesto de estos 63 carácteres:
['\n', '\r', ' ', '!', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


### Conversion de caracteres



In [83]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))


  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  "'" :   4,
  ',' :   5,
  '-' :   6,
  '.' :   7,
  ':' :   8,
  ';' :   9,
  '?' :  10,
  'A' :  11,
  'B' :  12,
  'C' :  13,
  'D' :  14,
  'E' :  15,
  'F' :  16,
  'G' :  17,
  'H' :  18,
  'I' :  19,
  'J' :  20,
  'K' :  21,
  'L' :  22,
  'M' :  23,
  'N' :  24,
  'O' :  25,
  'P' :  26,
  'Q' :  27,
  'R' :  28,
  'S' :  29,
  'T' :  30,
  'U' :  31,
  'V' :  32,
  'W' :  33,
  'X' :  34,
  'Y' :  35,
  'Z' :  36,
  'a' :  37,
  'b' :  38,
  'c' :  39,
  'd' :  40,
  'e' :  41,
  'f' :  42,
  'g' :  43,
  'h' :  44,
  'i' :  45,
  'j' :  46,
  'k' :  47,
  'l' :  48,
  'm' :  49,
  'n' :  50,
  'o' :  51,
  'p' :  52,
  'q' :  53,
  'r' :  54,
  's' :  55,
  't' :  56,
  'u' :  57,
  'v' :  58,
  'w' :  59,
  'x' :  60,
  'y' :  61,
  'z' :  62,


In [84]:
text_as_int = np.array([char2idx[c] for c in text])
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: "That, poor contempt, or claim'd thou slept so fait"
array([30, 44, 37, 56,  5,  2, 52, 51, 51, 54,  2, 39, 51, 50, 56, 41, 49,
       52, 56,  5,  2, 51, 54,  2, 39, 48, 37, 45, 49,  4, 40,  2, 56, 44,
       51, 57,  2, 55, 48, 41, 52, 56,  2, 55, 51,  2, 42, 37, 45, 56])


### Preparacion de los datos para el dataset

In [85]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

"That, poor contempt, or claim'd thou slept so faithful,\r\nI may contrive our father; and, in their def"
'eated queen,\r\nHer flesh broke me and puttance of expedition house,\r\nAnd in that same that ever I lame'
'nt this stomach,\r\nAnd he, nor Butly and my fury, knowing everything\r\nGrew daily ever, his great stren'
'gth and thought\r\nThe bright buds of mine own.\r\n\r\nBIONDELLO:\r\nMarry, that it may not pray their patien'
"ce.'\r\n\r\nKING LEAR:\r\nThe instant common maid, as we may less be\r\na brave gentleman and joiner: he that"
' finds us with wax\r\nAnd owe so full of presence and our fooder at our\r\nstaves. It is remorsed the bri'
"dal's man his grace\r\nfor every business in my tongue, but I was thinking\r\nthat he contends, he hath r"
"espected thee.\r\n\r\nBIRON:\r\nShe left thee on, I'll die to blessed and most reasonable\r\nNature in this h"
'onour, and her bosom is safe, some\r\nothers from his speedy-birth, a bill and as\r\nForestem with Richar'
"d in your heart\r

In [86]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "That, poor contempt, or claim'd thou slept so faithful,\r\nI may contrive our father; and, in their de"
Target data: "hat, poor contempt, or claim'd thou slept so faithful,\r\nI may contrive our father; and, in their def"


In [87]:
print (dataset)

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


### Batch size

In [88]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


### Ahora si la red neuronal

In [89]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [90]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [91]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [92]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [93]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 63) # (batch_size, sequence_length, vocab_size)


In [94]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [95]:
sampled_indices_characters

array([53, 35, 30, 18, 50,  5, 51, 48, 11, 42, 17, 22,  0,  8,  7, 19,  5,
       51, 61,  8, 44, 27,  1, 59, 40,  8, 12, 56,  3, 49, 37, 41, 43, 51,
       25, 12, 42, 36, 25, 47, 52, 54, 16,  0, 30, 18, 51, 22, 55, 37, 14,
       27, 37,  7, 62,  7, 42, 40, 43, 37, 47, 51, 59, 58, 56, 53,  9, 55,
       43, 44, 22,  1, 24,  3, 36,  6, 52, 39, 35, 52, 50, 48, 39, 47, 38,
       36,  5, 52, 23, 28, 17, 56,  8, 35, 43,  3, 61, 34,  1, 10])

## Función de entrenamiento

In [96]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [97]:
model.compile(optimizer='adam', loss=loss)

In [98]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [99]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
15/15 [==============================] - 5s 175ms/step - loss: 3.5677
Epoch 2/100
15/15 [==============================] - 2s 97ms/step - loss: 3.2379
Epoch 3/100
15/15 [==============================] - 2s 96ms/step - loss: 3.0283
Epoch 4/100
15/15 [==============================] - 1s 83ms/step - loss: 2.6575
Epoch 5/100
15/15 [==============================] - 2s 87ms/step - loss: 2.4214
Epoch 6/100
15/15 [==============================] - 1s 86ms/step - loss: 2.3091
Epoch 7/100
15/15 [==============================] - 1s 87ms/step - loss: 2.2404
Epoch 8/100
15/15 [==============================] - 2s 92ms/step - loss: 2.1802
Epoch 9/100
15/15 [==============================] - 1s 85ms/step - loss: 2.1259
Epoch 10/100
15/15 [==============================] - 2s 88ms/step - loss: 2.0746
Epoch 11/100
15/15 [==============================] - 2s 87ms/step - loss: 2.0255
Epoch 12/100
15/15 [==============================] - 2s 88ms/step - loss: 1.9777
Epoch 13/100
15/15 [====

### Resultados generacion de texto

In [100]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_100'

In [101]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [102]:
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [103]:
print(generate_text(model, start_string=u"a"))

all not make no long
Is trumper I would be coneruction with their own counsel then might come to thee.

TROILUS:
If it be, I have always find thee
There's not a woman's loyal deserving name shoes it!

Second Servant:
Sailors Hector; you shall be made;
And so my lordship says a credit shall be such
As moved to take my action, wonder well profare dective a hus ready might all but lift post instrument:
I am not Julius' heart, poor sons, lamented for her,
Or thine enemies you are lost another.

ULYSSES:
Thou toldst how now, kneel when I have, and mountain,
Hear the great it detection and thy quarrel I have seen
An elder suit of paragon.

TROILUS:
The gods I did.

Officer:
How now, volume, which made the first device out of a glass
Of himself that the bloody treasure of the court,
I see the faurter thee?

Second Neris:
Now, blessed France, and with thy speech can know?

ANTIPHOLUS OF EPHESUS:
A marriage, if I live,
I'll come.

Second Lord:

GLOUCESTER:
Ho
